# Part 1: Diabetes

In this part of the assignment, you will build a predictive model for diabetes disease progression in the next year based on current observed features of disease symptoms. 

**Learning objectives.** You will:
1. Train and test a linear model using ordinary least squares regression. 
2. Apply regularization, specifically LASSO, to build a sparse model.

The following code will download and preview three examples of the data. The ten features are as follows (in order):

- age age in years
- sex
- bmi body mass index
- bp average blood pressure
- s1 tc, total serum cholesterol
- s2 ldl, low-density lipoproteins
- s3 hdl, high-density lipoproteins
- s4 tch, total cholesterol / HDL
- s5 ltg, log of serum triglycerides level
- s6 glu, blood sugar level

The target value is a quantiative measure of disease progression after 1 year, where larger numbers are worse.

The code stores the feature matrix `X` as a two-dimensional NumPy array where each row corresponds to a data point and each column is a feature. The target value is stored as a one-dimensional NumPy array `y` where the index `i` element of `y` correpsonds to the row `i` data point of `X`.

Your overall goal in this part is to build and evaluate a linear model to predict the target variable `y` as a function of the ten features in `X`, and to identify which features are more significant for predicting `y`.

In [1]:
# Run but DO NOT MODIFY this code

from sklearn.datasets import load_diabetes

# Load the diabetes dataset
diabetes = load_diabetes(scaled = False)
print(diabetes.feature_names)

# Get the feature data and target variable
X = diabetes.data
y = diabetes.target

# Preview the first 3 data points
print(X[:3])
print(y[:3])

['age', 'sex', 'bmi', 'bp', 's1', 's2', 's3', 's4', 's5', 's6']
[[ 59.       2.      32.1    101.     157.      93.2     38.       4.
    4.8598  87.    ]
 [ 48.       1.      21.6     87.     183.     103.2     70.       3.
    3.8918  69.    ]
 [ 72.       2.      30.5     93.     156.      93.6     41.       4.
    4.6728  85.    ]]
[151.  75. 141.]


## Task 1

Randomly split the input data into a [train and test partition](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html), with 30% of the data reserved for testing. Use a random seed of `2024` for reproducibility of the results.

In [2]:
from sklearn.model_selection import train_test_split
# !pip install --upgrade scikit-learn 
# keep this here just in case, jupyter seems to think that I don't have RMSE sometimes

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=2024)

## Task 2

Build a baseline prediction by computing the [average](https://numpy.org/doc/stable/reference/generated/numpy.mean.html) target value of the training data. Evaluate the [root mean squared error](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.root_mean_squared_error.html#sklearn.metrics.root_mean_squared_error) between the baseline and the test data.

In [24]:
from sklearn.metrics import root_mean_squared_error as rmse
import numpy as np

avg_target = np.mean(y_train)
avg_targets = np.full_like(y_test, avg_target)
error = rmse(y_test, avg_targets)

print(f"Error: {error}")

Error: 78.17581726028506


## Task 3

Build a linear predictive model using [ordinary least squares regression](https://scikit-learn.org/stable/modules/linear_model.html#ordinary-least-squares) fit on the training data. 

Evaluate the [root mean squared error](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.root_mean_squared_error.html#sklearn.metrics.root_mean_squared_error) of the model on **both** the training data **and** the test data (that is, the training error and the generalization error). Report both and briefly discuss the results: Do you observe underfitting or overfitting?

Note that the model predictions on the test data may not be perfect, but they should improve meaningfully over the simple baseline from Task 2 or something is wrong.

In [23]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()
model.fit(X_train, y_train)
print(f"Coefficients: {model.coef_}")

y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)
 
training_error = rmse(y_train, y_train_pred)
generalization_error = rmse(y_test, y_test_pred)

print(f"Training error: {training_error}")
print(f"Generalization error: {generalization_error}")

Coefficients: [  0.10529823 -23.7197526    5.67267281   1.28413159  -1.14194878
   0.69859362   0.48121307   6.21245306  66.81556482   0.15845329]
Training error: 52.85235480118789
Generalization error: 55.61674711723442


The error is significantly less than it was when we used the average. There's also a relatively small generalization gap. 

## Task 4

If your goal is to understand which of the input features in `X` are most important for predicting the target `y`, the linear model you built in task 3 may not be very helpful. Build a new linear model using [Lasso regression](https://scikit-learn.org/stable/modules/linear_model.html#lasso) that achieves comparable generalization error as the task 3 model using ordinary least squares regression (within 10% of the root mean squared error on the test set), but with **0 for at least three of the model coefficients** (that is, the model does not use these features to make predictions). 

You may need to try multiple vaues of the `alpha` *hyperparameter* to find a model that satisfies both the error and *sparsity* constraints (that at least three of the coefficients are 0). Nevertheless, you should only evaluate error on the test dataset **once**. Show your work for how you find a good `alpha` in code and explain your work in English below. Standard approaches would be to split the training data into a train and validation set, or to use [cross validation](https://scikit-learn.org/stable/modules/cross_validation.html) on the training data.

For your final fit Lasso model with the chosen `alpha`, report the [root mean squared error](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.root_mean_squared_error.html#sklearn.metrics.root_mean_squared_error) on the test data. Also report the model coefficients and use this to explain which features (see their names/interpretations above) seem less important for predicting the target.

In [21]:
from sklearn.linear_model import Lasso
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer

rmse_scorer = make_scorer(rmse, greater_is_better=False)
possible_alphas = np.logspace(-3, 3, 20)

best_alpha = None
best_rmse = float("inf")
best_model = None

#basically, I am looping through possible alphas and using a cross validation test for each model constructed with each alpha.
#when the model meets our standards, it is overwritten as such
for alpha in possible_alphas:
    model = Lasso(alpha=alpha, random_state=2025, max_iter=10000) # increase max_iter for convergence
    
    validation_rmse_scores = cross_val_score(model, X_train, y_train, cv=5, scoring=rmse_scorer)
    validation_rmse = -np.mean(validation_rmse_scores) # have to negate cross val score output
    
    model.fit(X_train, y_train)
    num_zero_coefs = np.sum(model.coef_ == 0)
    
    if validation_rmse < best_rmse and num_zero_coefs >= 3:
        best_rmse = validation_rmse
        best_alpha = alpha
        best_model = model
        
print(f"Best Alpha: {best_alpha}")
print(f"Best RMSE: {best_rmse}")

coefficients = best_model.coef_
print(f"Model Coefficients: {coefficients}")

features = diabetes.feature_names
less_important_features = [features[i] for i in range(len(coefficients)) if coefficients[i] == 0]
print(f"Less important features: {less_important_features}")

Best Alpha: 6.158482110660261
Best RMSE: 56.0303515428855
Model Coefficients: [ 0.         -0.          5.90643131  1.25723314  1.09583832 -1.29098671
 -1.87854119  0.          0.          0.16136646]
Less important features: ['age', 'sex', 's4', 's5']


So it seems like age, sex, total cholesterol / HDL, and serum triglycerides level are the least important features. The RMSE yielded is around the same as the generalization RMSE from the linear regression model. 